In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoost, CatBoostRegressor, Pool
from catboost import CatBoostClassifier
from sklearn import datasets
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
import time

In [ ]:
test_new = pd.read_csv('test.tsv.gz',sep='\t',compression='gzip')

In [ ]:
train_new = pd.read_csv('train.tsv.gz',sep='\t',compression='gzip')

In [ ]:
lst = []
for name in train_new.columns:
    nu = len(train_new[name].unique())
    nisnan =  train_new[name].isnull().sum()
    mode = train_new[name].mode().values[0]
    lst.append([name, nu, nisnan,mode, train_new[name].unique()[:min(nu,10)]])
pd.DataFrame(lst,columns = ['column', 'nunique', 'nnull', 'mode','values'])

,column,nunique,nnull,mode,values
0,Unnamed: 0,29989753,0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
1,ts,259201,1,1.37943e+09,"[1379278800.0, 1379278801.0, 1379278802.0, 137..."
2,label,3,1,0,"[0.0, 1.0, nan]"
3,A,968188,1,4.01307e+09,"[2733540231.0, 1964843810.0, 660357674.0, 2962..."
4,B,88651,1,2.27719e+09,"[3500392421.0, 1232327635.0, 2066571765.0, 237..."
5,C,5931,1,2252,"[4454.0, 2081.0, 602.0, 4018.0, 4781.0, 5454.0..."
6,D,32205,1,21050,"[15573.0, 2547.0, 29581.0, 36969.0, 23457.0, 1..."
7,E,99,1,30,"[11.0, 29.0, 30.0, 35.0, 6.0, 10.0, 88.0, 83.0..."
8,F,3806,1,995,"[995.0, 1694.0, 3905.0, 551.0, 2038.0, 3418.0,..."
9,G,4,1,2,"[2.0, 0.0, 1.0, nan]"


In [ ]:
train_new['CG1_nan'] = train_new['CG1'].isna()
train_new['CG2_nan'] = train_new['CG2'].isna()
train_new['CG3_nan'] = train_new['CG3'].isna()

test_new['CG2_nan'] = test_new['CG2'].isna()
test_new['CG1_nan'] = test_new['CG1'].isna()
test_new['CG3_nan'] = test_new['CG3'].isna()

In [ ]:
from datetime import datetime

years = []
months = []
days = []
hours = []
for i in train_new['ts'].values:
    if is_nan_or_zero(i) == 0:
        years.append(datetime.fromtimestamp(i).year)
        months.append(datetime.fromtimestamp(i).month)
        days.append(datetime.fromtimestamp(i).day)
        hours.append(datetime.fromtimestamp(i).hour)
    else:
        years.append(i)
        months.append(i)
        days.append(i)
        hours.append(i)



years_test_new = []
months_test_new = []
days_test_new = []
hours_test_new = []
for i in test_new['ts'].values:
    if is_nan_or_zero(i) == 0:
        years_test_new.append(datetime.fromtimestamp(i).year)
        months_test_new.append(datetime.fromtimestamp(i).month)
        days_test_new.append(datetime.fromtimestamp(i).day)
        hours_test_new.append(datetime.fromtimestamp(i).hour)
    else:
        years_test_new.append(i)
        months_test_new.append(i)
        days_test_new.append(i)
        hours_test_new.append(i)


train_new['year'] = years
train_new['month'] = months
train_new['day'] = days
train_new['hour'] = hours

test_new['year'] = years_test_new
test_new['month'] = months_test_new
test_new['day'] = days_test_new
test_new['hour'] = hours_test_new

In [ ]:
import math

def is_nan_or_zero(val):
    return math.isnan(val) or val == 0


In [ ]:
years_test_new = []
months_test_new = []
days_test_new = []
hours_test_new = []
for i in test_new['ts'].values:
    if is_nan_or_zero(i) == 0:
        years_test_new.append(datetime.fromtimestamp(i).year)
        months_test_new.append(datetime.fromtimestamp(i).month)
        days_test_new.append(datetime.fromtimestamp(i).day)
        hours_test_new.append(datetime.fromtimestamp(i).hour)
    else:
        hours_test_new.append(i)

In [ ]:
train_new = train_new[train_new['label'].isna() == 0]

In [ ]:
categorical_features_names = [ 'A',  'C', 'D', 'E',   'H',
       'I', 'J', 'CG1', 'CG2',   'K', 'L',
        'hour', ]

feats = ['ts', 'CG1','CG2',
       'A', 'B',  'D', 'E', 'F', 'G', 'H', 'I',
                           'l1', 'l2', 'K', 'L','CG2_nan', 'hour']

In [ ]:
for i in categorical_features_names:
    train_new[i] = train_new[i].astype(str)
    test_new[i] = test_new[i].astype(str)

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_new[feats], train_new[['label']],
                                                    stratify=train_new[['label']],
                                                    test_size=0.2)

In [ ]:
feats = [['ts', 'CG1','CG2',
       'A', 'B',  'D', 'E', 'F', 'G', 'H', 'I',
                           'l1', 'l2', 'K', 'L','CG2_nan', 'hour']]

[['ts',
  'CG1',
  'CG2',
  'A',
  'B',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'l1',
  'l2',
  'K',
  'L',
  'CG2_nan',
  'hour']]

In [ ]:
feats = ['ts', 'CG1','CG2',
       'A', 'B',  'D', 'E', 'F', 'G', 'H', 'I',
                           'l1', 'l2', 'K', 'L','CG2_nan', 'hour']

In [ ]:
for i in categorical_features_names:
    test_new[i] = test_new[i].astype(str)

In [ ]:
%%time
preds2 = model.predict_proba(test_new[feats])[:,1]

CPU times: user 2min 25s, sys: 22.2 s, total: 2min 48s
Wall time: 2min 16s


In [ ]:
l_z = train_new[train_new['label']==0]

In [ ]:
%%time

model = CatBoostClassifier(custom_metric= ['Logloss'], learning_rate=0.1, n_estimators=120,max_depth=7, cat_features=categorical_features_names)
model.fit(train_new[feats],train_new['label'])

0:	learn: 0.5991034	total: 4.81s	remaining: 9m 32s
1:	learn: 0.5220900	total: 8.55s	remaining: 8m 24s
2:	learn: 0.4578851	total: 15.4s	remaining: 10m
3:	learn: 0.4036419	total: 22.1s	remaining: 10m 40s
4:	learn: 0.3573299	total: 25.9s	remaining: 9m 55s
5:	learn: 0.3174610	total: 29s	remaining: 9m 10s
6:	learn: 0.2829116	total: 32.8s	remaining: 8m 49s
7:	learn: 0.2528041	total: 35.1s	remaining: 8m 10s
8:	learn: 0.2264490	total: 37s	remaining: 7m 36s
9:	learn: 0.2032895	total: 38.1s	remaining: 6m 58s
10:	learn: 0.1828718	total: 41.4s	remaining: 6m 50s
11:	learn: 0.1648261	total: 42.7s	remaining: 6m 24s
12:	learn: 0.1488323	total: 44.6s	remaining: 6m 7s
13:	learn: 0.1346307	total: 47.6s	remaining: 6m
14:	learn: 0.1219975	total: 51.1s	remaining: 5m 57s
15:	learn: 0.1107459	total: 53.6s	remaining: 5m 48s
16:	learn: 0.1007124	total: 56.6s	remaining: 5m 43s
17:	learn: 0.0917636	total: 58s	remaining: 5m 28s
18:	learn: 0.0837598	total: 1m 1s	remaining: 5m 26s
19:	learn: 0.0766033	total: 1m 4s	r

In [ ]:
preds_neww = model.predict_proba(test_new[feats])[:,1]

In [ ]:
preds_ = model.predict_proba(X_test[feats])[:,1]

In [ ]:
preds_2 = model.predict_proba(X_test[feats])[:,1]

In [ ]:
preds_3 = model.predict_proba(X_test[feats])[:,1]

In [ ]:
log_loss(y_test,preds_)

0.014493850515968732

In [ ]:
log_loss(y_test,preds_3)

0.014741350383111898

In [ ]:
log_loss(y_test,preds_2)

0.01456536494045804

In [ ]:
log_loss(y_test,np.zeros(len(preds_)))

0.08787363014241754

In [ ]:
df = pd.DataFrame()
df['prediction'] = preds_neww

In [ ]:
df.to_csv('catboost_preds_new.csv',index=False)